In [5]:
import requests
import pandas as pd
url = 'https://donnees.montreal.ca/api/3/action/datastore_search?resource_id=c6f482bf-bf0f-4960-8b2f-9982c211addd'
response = requests.get(url)
d = response.json()

/tmp/ipykernel_148267/1775667198.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [7]:
df = pd.DataFrame(d['result']['records'])

In [8]:
df

,_id,CATEGORIE,DATE,QUART,PDQ,X,Y,LONGITUDE,LATITUDE
0,1,Vol de véhicule à moteur,2018-09-13,jour,30,294904.159001211,5047548.92099452,-73.626778,45.56778
1,2,Vol de véhicule à moteur,2018-04-30,jour,30,294904.159001211,5047548.92099452,-73.626778,45.56778
2,3,Vol de véhicule à moteur,2018-09-01,nuit,7,290274.565,5042150.474,-73.685928,45.519122
3,4,Méfait,2017-07-21,jour,21,None,None,None,None
4,5,Méfait,2017-07-29,jour,12,None,None,None,None
...,...,...,...,...,...,...,...,...,...
95,96,Méfait,2017-08-16,nuit,5,282109.687992077,5037403.00000111,-73.79022,45.476184
96,97,Méfait,2017-08-16,jour,3,277733.688002735,5038853.50000087,-73.84627,45.489081
97,98,Vol dans / sur véhicule à moteur,2017-08-17,nuit,33,294589.972003439,5043059.52100791,-73.63071,45.527378
98,99,Introduction,2018-09-25,jour,46,300206.96900415,5052306.49999714,-73.558887,45.610645
